### Trying to cluster

In [1]:
import functions
import numpy as np

In [2]:
# Folder containing the data
folder_path = "FC_matrices_times_wp11/"

# keep only ROIS
rois = [363, 364, 365, 368, 372, 373, 374, 377, 379, 361, 370, 362, 371, 10, 11, 12, 54, 56, 78, 96, 190, 191, 192, 234, 236, 258, 276, 8, 9, 51, 52, 53, 188, 189, 231, 232, 233]
rois = [roi - 1 for roi in rois]

t1_matrices, rsfMRI_full_info, rsfMRI_info, t1_subjects = functions.load_data_T1_only(folder_path, rois)
matrices, rsfMRI_full_info, rsfMRI_info, subjects = functions.load_data(folder_path, rois)

In [5]:
print("matrices shape: ", np.shape(matrices))
print("t1_matrices shape: ", np.shape(t1_matrices))

matrices shape:  ()
t1_matrices shape:  (22, 37, 37)


In [8]:
matrices = functions.matrices_to_wide_df(matrices)


AttributeError: module 'functions' has no attribute 'matrices_to_wide_df'